# T1.2 - Solution A
## Authors:
- Leonardo Kaplan 1212509
- Nino Fabrizio Tiriticco Lizardo 1113203

In [ ]:
# Pacotes usados
import pandas as pd # Para pegar os dados dos arquivos
from IPython.display import display # Para mostrar mais de uma informação em uma mesma célula
import ast # Para transformar string/object em estruturas de dados (listas, dicionários, ...)
import numpy as np # Para obter o total de valores por um atributo

# modelos de classificação
from sklearn import linear_model,datasets,svm,tree,neural_network
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier

#função de utilidade para separar teste e treinamento
from sklearn.model_selection import train_test_split

#funções de utilidade para metricas
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, precision_score, recall_score,f1_score

#funções de utilidade para seleção de features
from sklearn.feature_selection import VarianceThreshold,SelectKBest,chi2
import matplotlib.pyplot as plt
import seaborn as sns
import math

# isto é para usarmos markdown no meio de uma célula Python
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
import random
random.seed(1001001)

In [ ]:
# Carregando dados de cada um dos arquivos
data = pd.read_csv('in/winequality-red.csv',sep=';')
data.describe()

In [ ]:
# plotando combinações de features. Foi útil para vermos que não existem dependências ou independências claras.
#sns.set(style="ticks")
#sns.pairplot(data,hue="quality")
#plt.show()

In [ ]:
# mapa de correlações. útil para vermos que features devem ser mais 'pesadas' ou excluídas
plt.subplots(figsize=(10,10))
sns.heatmap(data.corr(),vmax=1,square=True,annot=True,cmap='Blues')
plt.show()

In [ ]:
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
X = data[features]
Y = data['quality']

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=random.randrange(0,100),test_size=0.25)

In [ ]:
# função para apresentar resultados de uma estimativa. Mostra o valor estimado (em azul) sobre o valor real (em vermelho).
def present(x,y,h):
    fig = plt.figure(figsize = (15,15))
    size = len(data.columns)
    for i in range(0,size-1):
        ax = fig.add_subplot(math.ceil(math.sqrt(size-1)), math.ceil(math.sqrt(size-1)), i+1)
        ax.scatter(x.iloc[:,i], y,  color='red')
        ax.scatter(x.iloc[:,i], h, color='blue')
    plt.show()
    

In [ ]:
# Realiza um teste de modelo: treinando, gerando valores e avaliando diversas métricas. 
# Retornamos os dados para serem plotados com a present se preciso.
def test(regr,X_train,Y_train,X_test,Y_test):
    regr.fit(X_train, Y_train)
    Y_pred = regr.predict(X_test)
    metrics = {
        'mse': mean_squared_error(Y_test, Y_pred),
        'r2':r2_score(Y_test, Y_pred),
        'accuracy':accuracy_score(Y_test, Y_pred), 
        'precision':precision_score(Y_test, Y_pred, average='macro'), 
        'recall':recall_score(Y_test, Y_pred, average='macro'),
        'f1':f1_score(Y_test, Y_pred, average='macro')
    }
    return Y_pred,metrics#,confusion_matrix

In [ ]:
# Métodos escolhidos e configurações escolhidas. 
# Determinamos as configurações experimentando, utilizando a test e a present. (na célula abaixo)
# Não mantive um registro dos experimentos realizados, mas variamos na mão ou com iterações. 
# Acho que não muito por acaso, os valores default costumavam ser os mais ótimos
methods = [
        linear_model.LogisticRegressionCV(),
        svm.SVC(),
        linear_model.SGDClassifier(max_iter=1000,tol=1e-3,loss="huber"),
        NearestCentroid(),
        KNeighborsClassifier(n_neighbors=5),
        tree.DecisionTreeClassifier(),
        neural_network.MLPClassifier(solver='lbfgs')
    ]
#função que testa todos os métodos, retornando uma lista com as métricas
def test_all(X_train,Y_train,X_test,Y_test):
    metrics = []
    for method in methods:
        #print(method.__class__.__name__)
        #print()
        Y_pred,metric = test(method,X_train,Y_train,X_test,Y_test)
        #print()
        metrics = metrics + [metric]
    return metrics

In [ ]:
# Aqui variávamos o método escolhido para teste e na célula acima, as configurações
Y_pred,_ = test(methods[0],X_train,Y_train,X_test,Y_test)
present(X_test,Y_test,Y_pred)

In [ ]:
# Aqui foi um momento de exploração utilizando o KNN e vendo onde teriamos a melhor relação erro/acurácia.
# Dessa forma, acharíamos o número de clusters ótimo, o que indica uma boa direção para o número de features.
es = []
ss = []
xs = list(range(1,12))
for i in xs:
    Y_pred,met = test(KNeighborsClassifier(n_neighbors=i),X_train,Y_train,X_test,Y_test)
    es += [met['mse']]
    ss += [met['accuracy']]
plt.scatter(xs, es,  color='red')
plt.scatter(xs, ss,  color='green')
plt.show()
#present(X_test,Y_test,Y_pred)

In [ ]:
# Aqui vemos o impacto de cada feature na qualidade (não foi muito útil)
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
X = data[features]
Y = data['quality']
sns.pairplot(data,x_vars=features,y_vars='quality',kind='reg',size=7,aspect=0.5)
plt.show()

In [ ]:
# utilizamos diversos algoritmos de filtragem de feature automática.
X = data[features]
print(np.shape(X))
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
X1 = sel.fit_transform(X)
print(np.shape(X1))
X2 = SelectKBest(chi2, k=2).fit_transform(X, Y)
print(np.shape(X2))
# Este 5 veio devido ao KNN e ao resultado do variance threshold
X3 = SelectKBest(chi2, k=5).fit_transform(X, Y)
print(np.shape(X3))
#sns.pairplot(data,x_vars=features,y_vars='quality',kind='reg',size=7,aspect=0.5)
#plt.show()

In [ ]:
#criamos 3 sub-datasets baseado nas filtragens
rand = random.randrange(0,100)
X1_train,X1_test,Y1_train,Y1_test=train_test_split(X1,Y,random_state=rand,test_size=0.25)
X2_train,X2_test,Y2_train,Y2_test=train_test_split(X2,Y,random_state=rand,test_size=0.25)
X3_train,X3_test,Y3_train,Y3_test=train_test_split(X3,Y,random_state=rand,test_size=0.25)

In [ ]:
metric_results = [
    test_all(X_train,Y_train,X_test,Y_test),
    test_all(X1_train,Y1_train,X1_test,Y1_test),
    test_all(X2_train,Y2_train,X2_test,Y2_test),
    test_all(X3_train,Y3_train,X3_test,Y3_test)
]

In [ ]:
# função para exibir um dataframe estilizado baseado no que é importante para a métrica escolhida
def show(metric,views):
    data = {
        'no filtering':[x[metric] for x in metric_results[0]], 
        'variance (80%)':[x[metric] for x in metric_results[1]],
        'chi2 k = 2':[x[metric] for x in metric_results[2]],
        'chi2 k =5':[x[metric] for x in metric_results[3]]
    }
    tabela = pd.DataFrame(data=data,index=[method.__class__.__name__ for method in methods])
    printmd('# '+ metric)
    return tabela.style.applymap(views[0]).apply(views[1]).apply(views[2], axis=None)


In [ ]:
def color_big_red(val):
    color = 'red' if val > 1 else 'black'
    return 'color: %s' % color

def highlight_min(s):
    is_min = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_min]
def color_negative_red(val):
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

def highlight_total_max(data):
    attr = 'background-color: {}'.format('cyan')
    is_max = data == data.max().max()
    return pd.DataFrame(np.where(is_max, attr, ''),index=data.index, columns=data.columns)

def highlight_total_min(data):
    attr = 'background-color: {}'.format('cyan')
    is_min = data == data.min().min()
    return pd.DataFrame(np.where(is_min, attr, ''),index=data.index, columns=data.columns)

metrics_show = {
        'mse': (color_big_red,highlight_min,highlight_total_min),
        'r2':(color_negative_red,highlight_max,highlight_total_max),
        'accuracy':(color_negative_red,highlight_max,highlight_total_max), 
        'precision':(color_negative_red,highlight_max,highlight_total_max), 
        'recall':(color_negative_red,highlight_max,highlight_total_max),
        'f1':(color_negative_red,highlight_max,highlight_total_max)
}
metrics_show_keys = list(metrics_show.keys())
metrics_show_values = list(metrics_show.values())
def show_metric(i):
    return show(metrics_show_keys[i],metrics_show_values[i])

### Da forma que escrevi as funções, o melhor método do dataset está em amarelo, o melhor método/dataset está em ciano e valores absurdos (acima de 1 ou abaixo de 0 dependendo do método) estão em vermelho

In [ ]:
show_metric(0)

In [ ]:
show_metric(1)

In [ ]:
show_metric(2)

In [ ]:
show_metric(3)

In [ ]:
show_metric(4)

In [ ]:
show_metric(5)

# Conclusão: 
### Filtrar features não trouxe resultados melhores na maior parte das métricas, com exceção da acurácia (que muitos consideram importante (não sei se realmente muito mais que as outras)). Acredito que tenha tipo algum impacto sobre a performance em tempo e memória, mas não medi isso. É bem possível que estes impactos tenham sido bem marginais também.
### Sobre os métodos, os melhores foram redes neurais e árvore de decisão. Fico dividido sobre qual eu recomendaria para alguém. Fico tentado a me basear nas métricas de performance das redes porém acho que tenho mais noção do que está acontecendo no caso da árvore de decisão. 